In [2]:
# Load environment variables
import os
import re
import pinecone
import glob
import json, jsonlines
import uuid
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

True

In [3]:
# Check api keys
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))

In [4]:
# data_folder='../data/FEA/'
data_folder='../data/AMS/'
docs = glob.glob(data_folder+'*.pdf')   # Only get the PDFs in the directory

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)

# print(docs[-1])
# loader=PyPDFLoader(docs[-1])
# doc_pages=loader.load_and_split(text_splitter)

pages=[]
for doc in docs[-3:]:
    loader=PyPDFLoader(doc)
    doc_pages=loader.load_and_split(text_splitter)
    pages.extend(doc_pages)
    print('Processed: '+doc)

Processed: ../data/AMS\AMS_2018.pdf
Processed: ../data/AMS\AMS_2020.pdf
Processed: ../data/AMS\AMS_2022.pdf


In [16]:
# Tidy text up
pages_dict=[]
for page in pages:
    page.metadata['source']=os.path.basename(page.metadata['source'])   # Strip path
    page.metadata['page']=page.metadata['page']+1   # Pages are 0 based, update
    # Merge hyphenated words
    page.page_content=re.sub(r"(\w+)-\n(\w+)", r"\1\2", page.page_content)
    # Fix newlines in the middle of sentences
    page.page_content = re.sub(r"(?<!\n\s)\n(?!\s\n)", " ", page.page_content.strip())
    # Remove multiple newlines
    page.page_content = re.sub(r"\n\s*\n", "\n\n", page.page_content)

    # Format into canopy format
    dict_temp=dict(page)
    dict_temp_out={'id':page.metadata['source']+"_"+str(page.metadata['page'])+str(uuid.uuid4()),
                   'text':dict_temp.pop('page_content'),
                   'source':dict_temp['metadata']['source'],
                   'metadata':dict_temp['metadata']}
    pages_dict.append(dict_temp_out)

print(pages[0])

NameError: name 'dict_temp_out' is not defined

In [14]:
print(pages_dict[0])

{'metadata': {'source': 'AMS_2018.pdf', 'page': 3}, 'id': 'AMS_2018.pdf_3986a43b7-4795-4dd1-a81d-34b02d49b5f2', 'text': '44th Aerospace Mechanisms SymposiumNASA/CP—2018-219887 May 2018Edward A. Boesiger, Compiler Lockheed Martin Space Systems Company, Sunnyvale, California', 'source': 'AMS_2018.pdf'}


In [15]:
with jsonlines.open(data_folder+'ams_data.jsonl', mode='w') as writer:
    writer.write_all(pages_dict)